# **Проект 5. Компьютер говорит нет!**
Целью - построение модели МО для вторичных клиентов банка, для принятия решения о выдаче кредина на основе данных из БКИ и анкет заёмщиков.


**Описания полей**

* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия “хорошей” работы
* bki_request_cnt - количество запросов в БКИ
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - связь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - флаг дефолта по кредиту

### **Библиотеки**

In [1]:
from pandas import Series
import numpy as np
import pandas as pd
import pandas_profiling

import operator
import time
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler,MinMaxScaler, RobustScaler, OrdinalEncoder, PowerTransformer, QuantileTransformer
from sklearn.linear_model import LogisticRegression,Ridge
from sklearn.model_selection import StratifiedKFold, cross_val_score , train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from catboost import CatBoost,CatBoostClassifier, Pool
from catboost.utils import get_roc_curve
import lightgbm
from xgboost import XGBClassifier
import pickle

from sklearn.metrics import brier_score_loss, log_loss, confusion_matrix, plot_confusion_matrix, auc, mean_squared_error, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score,precision_recall_curve,average_precision_score

from itertools import combinations
from scipy.stats import ttest_ind
import statsmodels.api as sm
import scipy.stats as sst
import warnings
import plotly.express as px
import datatable as dt
from sklearn import metrics
import itertools

from datetime import date
from datetime import datetime, timedelta

import warnings; warnings.simplefilter('ignore') # Убирает некритические сообщения
RANDOM_SEED = 40                                 # Зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
!pip freeze > requirements.txt                   # Зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы

### 1. Загрузка набора данных

In [2]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [3]:
# Для корректной обработки данных объединим train и test в один датасет
train['sample'] = 1 # помечаем train
test['sample'] = 0 # помечаем test
test['default'] = 0 # т.к. значения дефолта отсутствуют для тестовой выборки, то пока
# заполним нулями

data = test.append(train, sort=False).reset_index(drop=True)

In [4]:
data.info()

# 2. Создадим наивную модель

In [6]:
#избавимся от пропусков
train_naive = train.dropna()

In [7]:
#удалим все признаки которые нуждаются в каком-либо преобразовании
train_naive.drop(['app_date','education','sex','car','car_type','foreign_passport'],axis=1,inplace=True)

In [8]:
train_naive.sample(8)

In [9]:
# функци метрик ошибок
def scores(y_test, y_pred, prob):
    fpr, tpr, threshold = roc_curve(y_test, prob)
    roc_auc = roc_auc_score(y_test, prob)
    
    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()
    print ("f1_score:",round(f1_score(y_test,y_pred), 3))
    print ("accuracy_score:",round(accuracy_score(y_test,y_pred), 3))
    print ("precision_score:",round(precision_score(y_test,y_pred), 3))
    print ("recall_score:",round(recall_score(y_test,y_pred), 3))
    print ("log_loss:",round(log_loss(y_test,y_pred), 3))
    print ("roc_auc_score:",round(roc_auc, 3))
    print("average_precision_score:", round(average_precision_score(y_test,y_pred), 3))
    print("brier_score_loss:", round(brier_score_loss(y_test,y_pred), 3))
    print('Confusion matrix:\n{}' .format(confusion_matrix(y_test,y_pred)))

In [10]:
#создаем модель на основе наивных данных
model_naive = LogisticRegression(max_iter = 1000)
X = train_naive.drop(['default','client_id'], axis = 1).values
y = train_naive['default'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=RANDOM_SEED)
model_naive.fit(X_train, y_train)
y_pred = model_naive.predict(X_test)
proba = model_naive.predict_proba(X_test)[:, 1]

In [11]:
scores(y_test, y_pred, proba)

### 3. Первичный анализ набора данных

In [12]:
pandas_profiling.ProfileReport(data)

**Предварительные наблюдения:**

* 110148 строк, в т.ч. 73799 строк в наборе для обучения и 36349 строк в наборе для тестирования
* дублирующиеся строки не обнаружены
* Education содержит пропуски
* 7 числовых переменных, 10 категориальных и 3 поля Boolean (включая целевую переменную и признак набора для обечения/теста)
* ни у одного из полей нет большой корреляции с целевой переменной
* есть значимая корреляция между Доходом заёмщика и рейтингом региона

### 4. Предобработка данных
#### 4.1 Обработка NaN

In [13]:
# Education, ординарные признаки
#Образование - это ординарный признак, т.к. есть увеличение уровня образования
data["education"].unique()

In [14]:
data['education'].value_counts(dropna=False)

In [15]:
# Посмотрим, какое образование является модой
data["education"].mode()

In [16]:
# заполним пропуски значением, которое встречается чаще всего
data["education"].fillna('SCH', inplace=True)

In [17]:
data.info()

#### 4.2 Обработка признаков

In [18]:
# Cоздадим списки по категориям датасета.
# Категориальные, бинарные, числовые.
# даты пока не будем включать в признак

bin_cols = ['sex', 'car', 'car_type', 'foreign_passport', 'good_work']
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
num_cols = ['age', 'decline_app_cnt', 'income', 'bki_request_cnt', 'score_bki']

In [19]:
# Для бинарных признаков мы будем использовать LabelEncoder

label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
    
data['education'] = label_encoder.fit_transform(data['education'])
# убедимся в преобразовании    
data.head()

In [20]:
# Разделим обратно на train и test
train = data.query('sample == 1').drop(['sample'], axis=1)
test = data.query('sample == 0').drop(['sample'], axis=1)

In [21]:
# удалим для X целевую переменную и client_id, app_date
X = train.drop(columns = ['default', 'client_id','app_date'])
Y = train['default']

In [29]:
#создаем модель на основе обработаннх данных
model = LogisticRegression(max_iter = 1000)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=RANDOM_SEED)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
proba = model.predict_proba(X_test)[:, 1]

In [30]:
scores(y_test, y_pred, proba)

**Видим что показатели не сильнго изменились, поэтому преобразовываем параметры дальше**

# Найдем коррелиции данных

In [31]:
# Посмотрим матрицу коэффициента корреляции численных признаком
sns.set(font_scale = 1)
fig, axis = plt.subplots(figsize=(12,12))
sns.heatmap(data[num_cols+cat_cols+bin_cols].corr().abs(), vmin=0, vmax=1, square=True,
           annot=True, fmt=".2f", linewidths=0.1, cmap="YlGnBu")

Замечаем, что большая корреляция между адресом работы и домашним адресом, автомобилем и его типом

In [32]:
#Уберем коррелирующие признаки из катеригорий и app_date
bin_cols = ['sex', 'car', 'foreign_passport', 'good_work']
cat_cols = ['education', 'region_rating', 'work_address', 'sna', 'first_time']
num_cols = ['age', 'decline_app_cnt', 'income', 'bki_request_cnt', 'score_bki']

In [33]:
data[bin_cols+cat_cols+num_cols].sample(8)

# Нормализуем данные

In [34]:
plt.figure()
sns.distplot(data['age'].apply(lambda x: np.log(x+1)))
data['age'] = data['age'].apply(lambda x: np.log(x+1))

In [35]:
plt.figure()
sns.distplot(data['income'].apply(lambda x: np.log(x+1)))
data['income'] = data['income'].apply(lambda x: np.log(x+1))

In [36]:
plt.figure()
sns.distplot(data['decline_app_cnt'].apply(lambda x: np.log(x+1)))
data['decline_app_cnt'] = data['decline_app_cnt'].apply(lambda x: np.log(x+1))

In [37]:
plt.figure()
sns.distplot(data['bki_request_cnt'].apply(lambda x: np.log(x+1)))
data['bki_request_cnt'] = data['bki_request_cnt'].apply(lambda x: np.log(x+1))

In [38]:
plt.figure()
sns.distplot(data['region_rating'].apply(lambda x: np.log(x+1)))
data['region_rating'] = data['region_rating'].apply(lambda x: np.log(x+1))

In [39]:
data.sample(8)

In [40]:
# Разделим обратно на train и test
train = data.query('sample == 1').drop(['sample'], axis=1)
test = data.query('sample == 0').drop(['sample'], axis=1)

In [41]:
# удалим для X целевую переменную и client_id, app_date, home_address, car_type
X = train.drop(columns = ['default', 'client_id','app_date','car_type','home_address'])
Y = train['default']

In [42]:
#создаем модель на основе обработаннх данных
model = LogisticRegression(max_iter = 1000)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=RANDOM_SEED)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
proba = model.predict_proba(X_test)[:, 1]

In [43]:
scores(y_test, y_pred, proba)

**После нормализации данных и удаление корреляционных параметров модель улучшилась**

#### Модель CatBoost